In [45]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras.optimizers import Adam
from keras.layers import Input, Lambda
from keras.models import Model
from keras import losses

In [46]:
#dir = './sequence/*'
dir = './latest_sequence/bfs/*'

In [47]:
# file read
all_data = []
sequence_length = []
name = []
for file in sorted(glob.glob(dir)):
    name.append(file.split('/')[-1].replace('.txt', ''))
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [48]:
x_train, x_test, train_name, test_name = train_test_split(all_data, name, test_size=0.3)
x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [49]:
train_name
tr_names= []
for name in train_name:
    tr_names.append(name.split('-')[0].replace('graph', ''))

In [50]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 500
steps_per_epoch = len(x_train)

In [51]:
def custom_loss(y_true, y_pred):
    loss1 = losses.mean_squared_error(y_true, y_pred)
    loss2 = losses.categorical_crossentropy(y_true, y_pred)
    return loss1 * 0.7 + loss2 * 0.3

In [52]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss=losses.mean_squared_error, optimizer='adam')
#lstm_autoencoder_500 = lstm_autoencoder

In [53]:
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0

In [ ]:
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1)

Epoch 1/500
2352/2352 [==============================] - 23s 10ms/step - loss: 35.9514
Epoch 2/500
2352/2352 [==============================] - 21s 9ms/step - loss: 34.9104
Epoch 3/500
2352/2352 [==============================] - 21s 9ms/step - loss: 34.8690
Epoch 4/500
2352/2352 [==============================] - 20s 9ms/step - loss: 41.4048
Epoch 5/500
2352/2352 [==============================] - 20s 9ms/step - loss: 35.0476
Epoch 6/500
2352/2352 [==============================] - 20s 9ms/step - loss: 19.6249
Epoch 7/500
2352/2352 [==============================] - 20s 9ms/step - loss: 13.1750
Epoch 8/500
2352/2352 [==============================] - 21s 9ms/step - loss: 11.2471
Epoch 9/500
2352/2352 [==============================] - 20s 8ms/step - loss: 10.4691
Epoch 10/500
2352/2352 [==============================] - 20s 9ms/step - loss: 10.0001
Epoch 11/500
2352/2352 [==============================] - 20s 9ms/step - loss: 9.2855
Epoch 12/500
2352/2352 [============================

2352/2352 [==============================] - 19s 8ms/step - loss: 0.4554
Epoch 97/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.4761
Epoch 98/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.4848
Epoch 99/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.5335
Epoch 100/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.5838
Epoch 101/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.8354
Epoch 102/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.8664
Epoch 103/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.4989
Epoch 104/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.4027
Epoch 105/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.4508
Epoch 106/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.6929
Epoch 107/500
2352/2352 [==============================] - 1

2352/2352 [==============================] - 19s 8ms/step - loss: 0.1480
Epoch 191/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.1312
Epoch 192/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.1640
Epoch 193/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.1370
Epoch 194/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.1454
Epoch 195/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.1359
Epoch 196/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.1426
Epoch 197/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.1337
Epoch 198/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.1352
Epoch 199/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.1371
Epoch 200/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.1431
Epoch 201/500
2352/2352 [==============================] 

2352/2352 [==============================] - 20s 8ms/step - loss: 0.0949
Epoch 285/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0915
Epoch 286/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0986
Epoch 287/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0889
Epoch 288/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.1127
Epoch 289/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0825
Epoch 290/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.1003
Epoch 291/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0803
Epoch 292/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0967
Epoch 293/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0916
Epoch 294/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0798
Epoch 295/500
2352/2352 [==============================] 

2352/2352 [==============================] - 20s 9ms/step - loss: 0.0734
Epoch 379/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.0778
Epoch 380/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.0708
Epoch 381/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.0663
Epoch 382/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0832
Epoch 383/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.0673
Epoch 384/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.0725
Epoch 385/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0682
Epoch 386/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0727
Epoch 387/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0773
Epoch 388/500
1376/2352 [================>.............] - ETA: 8s - loss: 0.0750

In [ ]:
model_json = lstm_autoencoder.to_json()
filename = input('filename: ')
with open('model/' + filename + '.json', 'w') as file:
    file.write(model_json)
lstm_autoencoder.save_weights('model/weights_' +  filename + '.h5')

In [47]:
loaded_model = model_from_json(open('model/' +filename + '.json').read())
loaded_model.load_weights('model/weights' + filename + '.h5')

In [44]:
mean= 0
for xt in x_test:
    xt = xt.reshape(1, xt.shape[0], xt.shape[1])
    out = loaded_model.predict(xt)
    mean += ((xt-out)**2).mean(axis=None)
print(mean/len(x_test))

1.9309339144058726


In [19]:
encoder = Model(loaded_model.input, loaded_model.layers[3].output)

In [20]:
test = x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1])
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])